In [ ]:
# Instala os pacotes definidos no arquivo de dependências do projeto
!pip install -r requirements.txt

In [ ]:
#SEGMENTAÇÃO E SUBSTITUIÇÃO DO CANAL ALFA POR BRANCO

import os
import time
from PIL import Image
from rembg import remove

#Função de segmentação com fundo transparente (RGBA)
def segment_leaf_rembg(image_path):
    input_img = Image.open(image_path).convert("RGB")
    output = remove(input_img)  # retorna imagem RGBA (com alpha)
    return output

#Diretórios
root_dir = r"C:\Users\Julia\Downloads\DataSet_Original"             #Diretório raiz com imagens originais em subdiretórios
save_root_rembg = r"C:\Users\Julia\Downloads\Segmented_original"    #Diretório onde as imagens com fundo transparente serão salvas
save_root_whitebg = r"C:\Users\Julia\Downloads\whitebg_original"    #Diretório onde as imagens com fundo branco serão salvas

os.makedirs(save_root_rembg, exist_ok=True)
os.makedirs(save_root_whitebg, exist_ok=True)

#Início do cronômetro geral
global_start = time.time()

#Percorre subpastas (classes)
for label in os.listdir(root_dir):
    class_path = os.path.join(root_dir, label)
    if not os.path.isdir(class_path):
        print(f"Pulado (não é diretório): {class_path}")
        continue

    print(f"\nProcessando classe: {label}")
    class_start = time.time()  # início da classe

    #Cria subpastas correspondentes nos diretórios de destino
    save_dir_rembg = os.path.join(save_root_rembg, label)
    save_dir_whitebg = os.path.join(save_root_whitebg, label)
    os.makedirs(save_dir_rembg, exist_ok=True)
    os.makedirs(save_dir_whitebg, exist_ok=True)

    #Conta imagens
    total_imgs = 0
    for img_name in os.listdir(class_path):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            total_imgs += 1

    #Percorre imagens da subpasta
    processed = 0
    for img_name in os.listdir(class_path):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            image_path = os.path.join(class_path, img_name)
            name, ext = os.path.splitext(img_name)

            try:
                #Segmenta (fundo transparente)
                result_rembg = segment_leaf_rembg(image_path)

                #Salva versão segmentada (com alpha)
                save_path_rembg = os.path.join(save_dir_rembg, f"{name}_rembg.png")
                result_rembg.save(save_path_rembg)

                #Cria versão com fundo branco (RGBA --> RGB)
                bg = Image.new("RGBA", result_rembg.size, (255, 255, 255, 255))
                whitebg = Image.alpha_composite(bg, result_rembg).convert("RGB")

                save_path_whitebg = os.path.join(save_dir_whitebg, f"{name}_whitebg.jpg")
                whitebg.save(save_path_whitebg, "JPEG", quality=95)

                processed += 1
                print(f"OK ({processed}/{total_imgs}): {img_name}")

            except Exception as e:
                print(f"[!] ERRO com {img_name}: {e}")

    #Tempo da classe
    class_end = time.time()
    class_duration = class_end - class_start
    print(f"----Classe '{label}' concluída em {class_duration/60:.2f} min ({class_duration:.1f} s)----")

#Tempo total
global_end = time.time()
total_duration = global_end - global_start
print("\n-----------------------------------------------------------------------------------------------")
print(f"[*] Segmentação total concluída.")
print(f"[*] Tempo total: {total_duration/60:.2f} minutos ({total_duration/3600:.2f} horas)")

In [ ]:
#EXTRAÇÃO DE FEATURES

import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import ConvNextImageProcessor, ConvNextForImageClassification
import pretrainedmodels
import pretrainedmodels.utils as utils
import timm
from torchvision import transforms as T
import os

#Classe responsável por carregar modelos e realizar a extração de características de imagens
class FeatureExtractor:
    def __init__(self, device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        #Carrega o modelo ConvNext pré-treinado
        self.convnext_processor = ConvNextImageProcessor.from_pretrained("facebook/convnext-large-224-22k-1k")
        self.convnext_model = ConvNextForImageClassification.from_pretrained(
            "facebook/convnext-large-224-22k-1k"
        ).to(self.device)

        #Remove a camada de classificação final para obter apenas o vetor de características
        self.convnext_model.classifier = torch.nn.Identity()
        self.convnext_model.eval()

    #Função para extrair o vetor de características de uma imagem usando o modelo ConvNext
    def extract_convnext(self, image: Image.Image) -> np.ndarray:
        inputs = self.convnext_processor(image, return_tensors="pt").to(self.device)
        with torch.no_grad():
            features = self.convnext_model(**inputs).logits
        return features.cpu().numpy().flatten()

In [ ]:
#Caminho onde estão organizadas as classes (cada pasta = uma classe)
root_dir = r"C:\Users\Julia\Downloads\whitebg"

extractor = FeatureExtractor()        #Inicializa o extrator de features
imgs_extraction = None                #Matriz final de características

print("Iniciando extração...")

#Arquivo onde será salvo o nome da imagem e a classe correspondente
with open("features_soja_convnext.txt", "w") as f:
    for label in os.listdir(root_dir):
        class_path = os.path.join(root_dir, label)
        if not os.path.isdir(class_path):
            print(f"Pulado (não é diretório): {class_path}")
            continue

        print(f"Processando classe: {label}")
        for img_name in os.listdir(class_path):
            if img_name.lower().endswith(("jpg", "bmp", "png")):
                img_path = os.path.join(class_path, img_name)
                try:
                    print(f"Lendo imagem: {img_name}")
                    image = Image.open(img_path).convert("RGB")
                    
                    #Extrai o vetor de características da imagem
                    feature_vector = extractor.extract_convnext(image)
                    
                    #Salva o nome do arquivo e sua respectiva classe
                    f.write(f"{img_name}:{label}\n")

                    #Concatena os vetores extraídos em uma matriz única
                    if imgs_extraction is None:
                        imgs_extraction = np.expand_dims(feature_vector, axis=0)
                    else:
                        imgs_extraction = np.concatenate([imgs_extraction, [feature_vector]], axis=0)
                except Exception as e:
                    print(f"[!] ERRO ao processar {img_path}: {e}")

#Salva a matriz final com os vetores em formato binário (.npy)
np.save("features_extracted.npy", imgs_extraction)

#Exibe estatísticas da extração
print(f"[*] Total de vetores extraídos: {imgs_extraction.shape[0]}")
print(f"[*] Shape final da matriz de features: {imgs_extraction.shape}")